# References

GDS allows defining the component once in memory and reference to that structure in other components.

![](https://i.imgur.com/D4qaJpi.png)

As you build complex circuit you will make circuits that combine reference to other simpler circuits. Adding a reference is like having a pointer to the other devices.

The GDSII specification allows the use of references, and similarly gdsfactory uses them (with the `add_ref()` function).  So what is a reference? Simply put:  **A reference does not contain any geometry. It only *points* to an existing geometry**.  

Say Alice has a ridiculously large polygon with 100 billion vertices in it that we'll call BigPolygon. It's huge, and she needs to use it in her design 250 times.  Well, a single copy of BigPolygon takes up 100GB of memory just by itself, so she doesn't want to make 250 copies of it. Since Alice is clever, she instead *references* the polygon 250 times.  Each reference only uses a few bytes of memory -- it only needs to know the memory address of BigPolygon and a few other things. In this way, she can keep one copy of BigPolygon and use it again and again wherever she needs to.


Let's start by making a blank geometry (`Component`) then adding a single polygon
to it. 

In [ ]:
import numpy as np
import pp


# Create a blank Component
p = pp.Component()

# Add a polygon
xpts = [0,0,5,6,9,12]
ypts = [0,1,1,2,2,0]
p.add_polygon([xpts,ypts])

# Quickplot the Component with the polygon in it
p

Now, pretend we're in the same position as Alice: We want to reuse this polygon
repeatedly but do not want to make multiple copies of it.  To do so, we need to
make a second blank `Component`, this time called `D`.  In this new Component we'll
*reference* our Component `P` which contains our polygon.

In [ ]:
c = pp.Component()             # Create a new blank Component
poly_ref = c.add_ref(p)  # Reference the Component "p" that has the polygon in it
c # Quickplot the reference-containing Component "c"

OK, well that seemed to work, but it also seems thoroughly useless!  It looks
like you just made a copy of your polygon -- but remember, you didn't actually
make a second polygon, you just made a reference (aka pointer) to the original
polygon.  Let's add two more references to `c`:

In [ ]:
poly_ref2 = c.add_ref(p)  # Reference the Component "p" that has the polygon in it
poly_ref3 = c.add_ref(p)  # Reference the Component "p" that has the polygon in it

c # Quickplot the reference-containing Component "c"

Now you have 3x polygons all on top of each other.  Again, this would appear
useless, except that you can manipulate each reference indepedently. Notice that
when you called `c.add_ref(p)` above, we saved the result to a new variable each
time (`poly_ref`, `poly_ref2`, and `poly_ref3`)?  You can use those variables to
reposition the references. 

In [ ]:
poly_ref2.rotate(15) # Rotate the 2nd reference we made 15 degrees
poly_ref3.rotate(30) # Rotate the 3rd reference we made 30 degrees
c

Now you're getting somewhere!  You've only had to make the polygon once, but you're
able to reuse it as many times as you want.

## Modifying the referenced geometry

What happens when you change the original geometry that the reference points to?  In your case, your references in
`c` all point to the Component `p` that with the original polygon.  Let's try
adding a second polygon to `p`.

First you add the second polygon and make sure `P` looks like you expect:

In [ ]:
# Add a 2nd polygon to "p"
xpts = [14,14,16,16]
ypts = [0,2,2,0]
p.add_polygon([xpts,ypts], layer = (1,0))
p

That looks good.  Now let's find out what happened to `c` that contains the
three references.  Keep in mind that you have not modified `c` or executed any
functions/operations on `c` -- all you have done is modify `p`.

In [ ]:
c

 **When you modify the original geometry, all of the
references automatically reflect the modifications.**  This is very powerful,
because you can use this to make very complicated designs from relatively simple
elements in a computation- and memory-efficienct way.

Let's try making references a level deeper by referencing `c`.  Note here we use
the `<<` operator to add the references -- this is just shorthand, and is
exactly equivalent to using `add_ref()`

In [ ]:
c2 = pp.Component()             # Create a new blank Component
d_ref1 = c2.add_ref(c)  # Reference the Component "c" that 3 references in it
d_ref2 = c2 << c        # Use the "<<" operator to create a 2nd reference to c
d_ref3 = c2 << c        # Use the "<<" operator to create a 3rd reference to c

d_ref1.move([20,0])
d_ref2.move([40,0])

c2

## Arrays of references

Sometimes it's convenient to make an array or grid of the same geometry.  For
that purpose, the GDSII spec allows you to define arrays of references.  In
gdsfactory, these are added with the `add_array()` function. Note that by GDSII
definition these arrays are on a fixed grid -- gdsfactory does however have more
flexible arrangement options if desired, see for example `grid()` and
`packer()`.

Let's make a new Component and put a big array of our Component `c` in it:

In [ ]:
c3 = pp.Component() # Create a new blank Component
d_ref1 = c3.add_array(c, columns = 6, rows = 3, spacing = [20, 15])  # Reference the Component "c" 3 references in it with a 3 rows, 6 columns array
c3

As you've seen you have two ways to add a reference to our component:

1. create the reference and add it to the component

In [ ]:
c = pp.Component('reference_sample')
w = pp.c.waveguide(width=0.6)
wr = w.ref()
c.add(wr)
c

2. or do it in a single line

In [ ]:
c = pp.Component('reference_sample_shorter_syntax')
wr = c << pp.c.waveguide(width=0.6)
c

in both cases you can move the reference `wr` after created

In [ ]:
c = pp.Component('two_references')
wr1 = c << pp.c.waveguide(width=0.6)
wr2 = c << pp.c.waveguide(width=0.6)
wr2.movey(10)
c


You can also add an array of references for periodic structures. Lets create a [Distributed Bragg Reflector](https://picwriter.readthedocs.io/en/latest/components/dbr.html)

In GDS, there's a type of structure called a "CellArray" which takes a cell and repeats it NxM times on a fixed grid spacing. For convenience, `Component` includes this functionality with the add_array() function.  Note that CellArrays are not compatible with ports (since there is no way to access/modify individual elements in a GDS cellarray) 

In [ ]:
@pp.cell
def dbr_cell(w1=0.5, w2=0.6, l1=0.2, l2=0.4, waveguide_function=pp.c.waveguide):
    c = pp.Component()
    c1 = c << waveguide_function(length=l1, width=w1)
    c2 = c << waveguide_function(length=l2, width=w2)
    c2.connect(port="W0", destination=c1.ports["E0"])
    c.add_port("W0", port=c1.ports["W0"])
    c.add_port("E0", port=c2.ports["E0"])
    return c


w1 = 0.5
w2 = 0.6
l1 = 0.2
l2 = 0.4
n = 3
waveguide_function = pp.c.waveguide
c = pp.Component('reference_array')
cell = dbr_cell(w1=w1, w2=w2, l1=l1, l2=l2, waveguide_function=waveguide_function)
cell

In [ ]:
c = pp.Component('DBR')
c.add_array(cell, columns=n, rows=1, spacing=(l1 + l2, 100))
c

Finally we need to add ports to the new component

In [ ]:
p0 = c.add_port("W0", port=cell.ports["W0"])
p1 = c.add_port("E0", port=cell.ports["E0"])
p1.midpoint = [(l1 + l2) * n, 0]
c

## Connect references

We have seen that once you create a reference you can manipulate the reference to move it to a location. Here we are going to connect that reference to a port. Remeber that we follow that a certain reference `source` connects to a `destination` port

In [ ]:
pp.c.bend_circular()

In [ ]:
c = pp.Component("sample_reference_connect")

mmi = c << pp.c.mmi1x2()
b = c << pp.c.bend_circular()

b.connect("W0", destination=mmi.ports["E1"])
c

## component_sequence

When you have repetitive connections you can describe the connectivity as an ASCII map

In [ ]:
import pp

In [ ]:
bend180 = pp.c.bend_circular180()
wg_heater = pp.c.waveguide_heater()
wg = pp.c.waveguide()

# Define a map between symbols and (component, input port, output port)
string_to_device_in_out_ports = {
    "A": (bend180, "W0", "W1"),
    "B": (bend180, "W1", "W0"),
    "H": (wg_heater, "W0", "E0"),
    "-": (wg, "W0", "E0"),
}

# Generate a sequence
# This is simply a chain of characters. Each of them represents a component
# with a given input and and a given output

sequence = "AB-H-H-H-H-BA"
component = pp.c.component_sequence(sequence, string_to_device_in_out_ports)
component.name = 'component_sequence'
component

As the sequence is defined as a string you can use the string operations to build complicated sequences

In [ ]:

@pp.cell
def test_cutback_phase(straight_length=100.0, bend_radius=10.0, n=2):
    bend180 = pp.c.bend_circular(radius=bend_radius, angle=180)
    pm_wg = pp.c.waveguide_pin(length=straight_length)
    wg_short = pp.c.waveguide(length=1.0)
    wg_short2 = pp.c.waveguide(length=2.0)
    wg_heater = pp.c.waveguide_heater(length=10.0)
    taper = pp.c.taper_strip_to_ridge()

    # Define a map between symbols and (component, input port, output port)
    string_to_device_in_out_ports = {
        "I": (taper, "1", "wg_2"),
        "O": (taper, "wg_2", "1"),
        "S": (wg_short, "W0", "E0"),
        "P": (pm_wg, "W0", "E0"),
        "A": (bend180, "W0", "W1"),
        "B": (bend180, "W1", "W0"),
        "H": (wg_heater, "W0", "E0"),
        "-": (wg_short2, "W0", "E0"),
    }

    # Generate a sequence
    # This is simply a chain of characters. Each of them represents a component
    # with a given input and and a given output

    repeated_sequence = "SIPOSASIPOSB"
    heater_seq = "-H-H-H-H-"
    sequence = repeated_sequence * n + "SIPO" + heater_seq
    return pp.c.component_sequence(sequence, string_to_device_in_out_ports)

In [ ]:
c = test_cutback_phase(n=1)
c

In [ ]:
c = test_cutback_phase(n=2)
c